In [ ]:
!pip install roboflow

# Get Dataset

In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="cJUunyRGd3YuAM3cnXbU")
project = rf.workspace("volleyballcourt").project("volleyball-court-keypoints-k6y7r")
version = project.version(1)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to volleyball-court-keypoints-1 in yolov8:: 100%|██████████| 1002/1002 [00:00<00:00, 6800.97it/s]


In [2]:
dataset.location

'/home/khalil/VAS/training/volleyball-court-keypoints-1'

In [3]:
import shutil

shutil.move('volleyball-court-keypoints-1/train',
            'volleyball-court-keypoints-1/volleyball-court-keypoints-1/train'
            )

shutil.move('volleyball-court-keypoints-1/test',
            'volleyball-court-keypoints-1/volleyball-court-keypoints-1/test'
            )

shutil.move('volleyball-court-keypoints-1/valid',
            'volleyball-court-keypoints-1/volleyball-court-keypoints-1/valid'
            )

'volleyball-court-keypoints-1/volleyball-court-keypoints-1/valid'

# Train

In [4]:
#cleaning cash from gpu
import torch
torch.cuda.empty_cache()


In [5]:
dataset.location

'/home/khalil/VAS/training/volleyball-court-keypoints-1'

In [6]:
!yolo task=detect mode=train model=yolov8s-pose.pt data={dataset.location}/data.yaml epochs=20 imgsz=640 batch=8 amp=False


WARNING ⚠️ conflicting 'task=detect' passed with 'task=pose' model. Ignoring 'task=detect' and updating to 'task=pose' to match model.
New https://pypi.org/project/ultralytics/8.3.39 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1650, 3721MiB)
engine/trainer: task=pose, mode=train, model=yolov8s-pose.pt, data=/home/khalil/VAS/training/volleyball-court-keypoints-1/data.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dn

# Evaluate 

In [3]:
from ultralytics import YOLO

### on validation set

In [ ]:
# Load the trained model
model = YOLO('runs/pose/train5/weights/best.pt')

# Validate the model on the validation set
metrics = model.val(data='/home/khalil/VAS/training/volleyball-court-keypoints-1/data.yaml')
print(metrics)  

### on test set

In [ ]:
# Validate the model on the test set 
metrics = model.val(data='/home/khalil/VAS/training/volleyball-court-keypoints-1/data.yaml',split='test')
print(metrics)  

# Inference

### Predections results

In [5]:
import cv2
from matplotlib import pyplot as plt

# Run YOLO inference
results = model('../image.png')  
print(results[0].boxes.data)


image 1/1 /home/khalil/VAS/training/../image.png: 352x640 1 volleyball-court, 36.7ms
Speed: 1.5ms preprocess, 36.7ms inference, 4.3ms postprocess per image at shape (1, 3, 352, 640)
tensor([[1.7200e+02, 3.8500e+02, 1.5080e+03, 7.4200e+02, 8.8636e-01, 0.0000e+00]], device='cuda:0')


In [8]:
import cv2
from matplotlib import pyplot as plt

# Run YOLO inference
results = model('../1.png')  
print(results[0].boxes.data[0].cpu().numpy())


image 1/1 /home/khalil/VAS/training/../1.png: 640x640 1 volleyball-court, 21.0ms
Speed: 18.1ms preprocess, 21.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
[          0          10         578         468     0.26616           0]


In [ ]:
print(results)

### Court Detection

In [17]:
import cv2
from matplotlib import pyplot as plt

image_path = '../image1.jpg'

# Run YOLO inference
results = model(image_path)
print(results[0].boxes.data)

# Extract the first bounding box details
bbox_data = results[0].boxes.data[0].cpu().numpy()  # Convert tensor to numpy
x_min, y_min, x_max, y_max, confidence, class_id = bbox_data

# Load the image

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Draw the bounding box
x_min, y_min, x_max, y_max = map(int, [x_min, y_min, x_max, y_max])
label = f"volleyball-court {confidence:.0%}"
color = (255, 255, 0)  # Yellow color in RGB

# Draw rectangle
cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 3)

# Add label
font_scale = 1
font_thickness = 2
font = cv2.FONT_HERSHEY_SIMPLEX
text_size, _ = cv2.getTextSize(label, font, font_scale, font_thickness)
text_w, text_h = text_size
cv2.rectangle(image, (x_min, y_min - text_h - 10), (x_min + text_w, y_min), color, -1)
cv2.putText(image, label, (x_min, y_min - 5), font, font_scale, (0, 0, 0), font_thickness)

# Save the output image
output_path = '../bbox_output.png'
cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

# Show the image
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis("off")
plt.show()

print(f"Image saved as {output_path}")


image 1/1 /home/khalil/VAS/training/../image1.jpg: 448x640 1 volleyball-court, 29.0ms
Speed: 20.1ms preprocess, 29.0ms inference, 3.4ms postprocess per image at shape (1, 3, 448, 640)
tensor([[  0.0000, 147.0000, 492.0000, 333.0000,   0.8488,   0.0000]], device='cuda:0')


<Figure size 1000x800 with 1 Axes>

Image saved as ../bbox_output.png


### Keypoints visualisation 

In [ ]:
import cv2
from matplotlib import pyplot as plt

# Run YOLO inference
results = model('../image1.jpg')  # Perform inference
result = results[0]  # Extract the first result
keypoints = result.keypoints.data  # Get the keypoints as a tensor

# Load the image
img = cv2.imread('../image1.jpg')
if img is None:
    raise FileNotFoundError("Image not found. Check the file path.")

# Initialize list to store filtered keypoints
filtered_keypoints = []

# Plot keypoints on the image
if keypoints is not None:
    keypoints = keypoints.cpu().numpy()  # Move tensor to CPU and convert to numpy
    for person_keypoints in keypoints:
        for i, keypoint in enumerate(person_keypoints):
            x, y, confidence = keypoint  # Extract x, y, confidence
            if confidence > 0.5:  # Only process high-confidence keypoints
                filtered_keypoints.append((x, y, confidence))  # Add to filtered list
                print(f"Drawing keypoint {i} at ({x:.2f}, {y:.2f}) with confidence {confidence:.2f}")  # Debug
                cv2.circle(img, (int(x), int(y)), 10, (0, 255, 0), -1)  # Draw green keypoints
                cv2.putText(img, f"{i}:{confidence:.2f}", (int(x) + 10, int(y) + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)  # Add labels

# Save the processed image
cv2.imwrite('../keypoints_output_image.png', img)
print("Processed image saved as '../keypoints_output_image.png'")

# Display the image
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# Print filtered keypoints for debugging or further processing
print("Filtered Keypoints (x, y, confidence):")
for kpt in filtered_keypoints:
    print(kpt)
